<a href="https://colab.research.google.com/github/Neel7317/Generative-Models/blob/main/full_finetune_distilbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import torch
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments

# Sample Dataset
data = [
    {"text": "The soccer team clinched a thrilling victory in overtime.", "label": 0},
    {"text": "The cricket championship finals attracted global attention.", "label": 0},
    {"text": "A new record was set in the 100-meter sprint.", "label": 0},
    {"text": "Tennis legends face off in a historic grand slam final.", "label": 0},
    {"text": "The basketball league announces the upcoming season schedule.", "label": 0},
    {"text": "Fans cheer as their favorite football club wins the title.", "label": 0},
    {"text": "Olympic gold medalist announces retirement from the sport.", "label": 0},
    {"text": "The marathon saw record participation this year.", "label": 0},
    {"text": "Baseball's star player hits a remarkable home run.", "label": 0},
    {"text": "The Formula 1 race witnessed a dramatic last-lap overtake.", "label": 0},
    {"text": "New training facilities unveiled for the national hockey team.", "label": 0},
    {"text": "The underdog team surprises everyone with a stunning win.", "label": 0},
    {"text": "The swimming championship showcased extraordinary talent.", "label": 0},
    {"text": "The coach shares strategies ahead of the semi-finals.", "label": 0},
    {"text": "Athletes gear up for the international sports meet.", "label": 0},
    {"text": "The volleyball team secures a crucial win in the tournament.", "label": 0},
    {"text": "The government unveils a new healthcare policy.", "label": 1},
    {"text": "The president addresses the nation on economic reforms.", "label": 1},
    {"text": "The election results spark debates across the country.", "label": 1},
    {"text": "Parliament passes a bill to boost renewable energy.", "label": 1},
    {"text": "Diplomatic talks between the two nations yield progress.", "label": 1},
    {"text": "The mayor announces plans for urban development.", "label": 1},
    {"text": "The opposition criticizes the new tax policies.", "label": 1},
    {"text": "Campaigns heat up as candidates rally for voter support.", "label": 1},
    {"text": "The new law aims to strengthen cybersecurity measures.", "label": 1},
    {"text": "The prime minister visits a neighboring country for trade talks.", "label": 1},
    {"text": "Protests erupt against the controversial policy decision.", "label": 1},
    {"text": "A landmark decision by the court shapes future legislation.", "label": 1},
    {"text": "The minister highlights plans to improve education systems.", "label": 1},
    {"text": "National security is at the forefront of the political agenda.", "label": 1},
    {"text": "The governor outlines disaster relief efforts after the floods.", "label": 1},
    {"text": "A major treaty is signed to address climate change issues.", "label": 1},
    {"text": "Students prepare for university entrance exams.", "label": 2},
    {"text": "The new study techniques promise better learning outcomes.", "label": 2},
    {"text": "The education board announces changes in the curriculum.", "label": 2},
    {"text": "A groundbreaking study explores the effects of online learning.", "label": 2},
    {"text": "A new scholarship program supports underprivileged students.", "label": 2},
    {"text": "Teachers attend workshops on integrating technology in classrooms.", "label": 2},
    {"text": "The library adds hundreds of new academic resources.", "label": 2},
    {"text": "Researchers publish findings on improving memory retention.", "label": 2},
    {"text": "Study groups form to tackle challenging science topics.", "label": 2},
    {"text": "Students collaborate on innovative projects for the science fair.", "label": 2},
    {"text": "The school introduces coding classes for young learners.", "label": 2},
    {"text": "An education conference discusses the future of AI in learning.", "label": 2},
    {"text": "Experts share tips for mastering difficult math concepts.", "label": 2},
    {"text": "The university announces free courses on climate studies.", "label": 2},
    {"text": "A recent survey highlights the need for mental health support in schools.", "label": 2},
    {"text": "The tutoring center reports an increase in attendance.", "label": 2},
    {"text": "Students celebrate as their project wins the national innovation award.", "label": 2},
    {"text": "A new app helps students track their study habits effectively.", "label": 2},
]

# Split the data into training and validation sets
train_texts, val_texts = train_test_split(data, test_size=0.2, random_state=42)
train_dataset = Dataset.from_list(train_texts)
val_dataset = Dataset.from_list(val_texts)

# Tokenizer and Model
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

# Tokenization
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)

# Remove unneeded columns
tokenized_train = tokenized_train.remove_columns(["text"])
tokenized_val = tokenized_val.remove_columns(["text"])

# Format to Torch
tokenized_train.set_format("torch")
tokenized_val.set_format("torch")

# Training Arguments
training_args = TrainingArguments(
    output_dir="./results_full_finetune_distilbert",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs_full_finetune_distilbert",
    logging_steps=10,
    no_cuda=True,  # Ensure training on CPU
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Save the model and tokenizer
model.save_pretrained("./full_finetune_distilbert_model")
tokenizer.save_pretrained("./full_finetune_distilbert_model")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,1.060100,1.019964
2,0.917500,0.926786
3,0.809200,0.879327


('./full_finetune_distilbert_model/tokenizer_config.json',
 './full_finetune_distilbert_model/special_tokens_map.json',
 './full_finetune_distilbert_model/vocab.txt',
 './full_finetune_distilbert_model/added_tokens.json')

In [4]:
import numpy as np
from sklearn.metrics import accuracy_score

# Evaluate the model
results = trainer.evaluate()
print(f"Evaluation Results: {results}")

# Make predictions on the validation set
predictions = trainer.predict(tokenized_val)

# Convert logits to predicted labels
predicted_labels = np.argmax(predictions.predictions, axis=-1)

# Convert true labels to a NumPy array
true_labels = np.array(tokenized_val["label"])

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Validation Accuracy: {accuracy}")


Evaluation Results: {'eval_loss': 0.8793267011642456, 'eval_runtime': 22.7577, 'eval_samples_per_second': 0.439, 'eval_steps_per_second': 0.132, 'epoch': 3.0}
Validation Accuracy: 1.0
